In [12]:
# ver04
# image 변주를 주고, smote로 해봄 acc 97.5 f1 95? 나오지만 정말 이상한 데이터들은 못찾음
# filename	answer	비슷	pred	sum	
# 00198.png	18	11,28	11	28	가려진거
# 00829.png	27	30	30	27	절반 그림자
# 00072.png	30	28,31	11	25	흐림
# 01079.png	21	14	41	15	배경다르고 가운데 모양만있음
# 02635.png	5	4	2	13	너무 흐림
# 00033.png	23	31	31	12	안보임
# 00790.png	2	31	3	11	어두움
# 00050.png	25	11	2	9	가려지고 너무흐림
					
					
# 8297					말도안됨
	
# 					가운데만 Crop해보고 시도하고
# 					해상도 올리는걸 Test에도?
# 0601 22 Test data 해상도만 올려서 해보자

# data_transforms_rand = transforms.Compose([
#   transforms.RandomRotation(degrees=15),
#   transforms.RandomAutocontrast(),
#   transforms.RandomAdjustSharpness(4),
#   transforms.Resize([112, 112]),
#   transforms.Pad(10),
#   transforms.ToTensor()
#   # transforms.Normalize(0, 1)
#   ])

In [13]:
_ = """
+ CutMix 
+ data_aug

"""

In [14]:
# traffic sign down
!gdown --id 1ysKr2SJAxGWrdSvoA5fn40jhRQmbPswl
!gdown --id 1-MsSMJqE8AdMQYgS_tiYiaeTkiF4WB5F
!unzip data.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1ysKr2SJAxGWrdSvoA5fn40jhRQmbPswl 

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-MsSMJqE8AdMQYgS_tiYiaeTkiF4WB5F
To: /content/Test_Label.csv
100% 118k/118k [00:00<00:00, 84.1MB/s]
Archive:  data.zip
replace Train/0/00000_00000_00000.png? [y]es, [n]o, [A]ll, [N]one, [

In [15]:
!ls

data.zip  sample_data  Test  Test_Label.csv  Train


In [16]:
# Few imports

import torch
import torchvision

from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch.utils.data as data
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn as nn
from torchsummary import summary
from __future__ import print_function

import zipfile
import time
import numpy as np
import os
import matplotlib.pyplot as plt
import random
import argparse
import pandas as pd
import shutil
from PIL import Image

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import f1_score

In [17]:
# # https://pytorch.org/docs/stable/notes/randomness.html

# # REPRODUCIBILITY
# # Controlling sources of randomness
def set_seed(RANDOM_SEED=333):
    torch.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED) # if use multi-GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(RANDOM_SEED)
    random.seed(RANDOM_SEED)
    os.environ['PYTHONHASHEED'] = str(RANDOM_SEED)
    
set_seed()

# torch.manual_seed(RANDOM_SEED)
# torch.cuda.manual_seed_all(RANDOM_SEED)

# random.seed(RANDOM_SEED)
# np.random.seed(RANDOM_SEED)

# def seed_worker(worker_id):
#     worker_seed = torch.initial_seed() % 2**32
#     np.random.seed(worker_seed)
#     random.seed(worker_seed)



# # reduced performance
# # torch.backends.cudnn.deterministic = True # torch.use_deterministic_algorithms(True)
# # torch.backends.cudnn.benchmark = False

In [18]:
# Before starting, clear the memory

torch.cuda.empty_cache()

In [19]:
# Define path of training data
train_data_path = "./Train"
data_transformer_stat = transforms.Compose([transforms.ToTensor()])
train_data_stat = torchvision.datasets.ImageFolder(root = train_data_path, transform = data_transformer_stat)
print(f"Number of training samples = {len(train_data_stat)}")

Number of training samples = 26010


In [ ]:
# calculate the mean and standard deviation of train_data_stat

meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x,_ in train_data_stat]
stdRGB = [np.std(x.numpy(), axis=(1,2)) for x,_ in train_data_stat]

meanR = np.mean([m[0] for m in meanRGB])
meanG = np.mean([m[1] for m in meanRGB])
meanB = np.mean([m[2] for m in meanRGB])

stdR = np.mean([s[0] for s in stdRGB])
stdG = np.mean([s[1] for s in stdRGB])
stdB = np.mean([s[2] for s in stdRGB])

print(meanR, meanG, meanB)
print(stdR, stdG, stdB)

In [ ]:
# Define the transformations. To begin with, we shall keep it minimum - only resizing the images and converting them to PyTorch tensors

data_transforms = transforms.Compose([
    transforms.Resize([48, 48]),
    transforms.ToTensor(),
    transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
    ])

# # Define transformations

data_transforms_rand = transforms.Compose([
  transforms.RandomOrder([
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        transforms.RandomAffine(20, shear=20),
        transforms.RandomAutocontrast(),
        transforms.RandomAdjustSharpness(4),
        transforms.RandomErasing(p=0.05, scale=(0.02, 0.33), value=0),
        transforms.GaussianBlur(kernel_size=(3, 3), sigma=(1, 3))
    ], p=0.7)
  ]),
  # transforms.CenterCrop(112*0.8),
  # transforms.Resize([112, 112]),
  transforms.Resize([48, 48]),
  transforms.ToTensor(),
  transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
])

data_transforms_rand2 = transforms.Compose([
  transforms.Grayscale(num_output_channels=3), 
  transforms.Resize([48, 48]),  
  # transforms.CenterCrop(112*0.8),
  # transforms.Resize([112, 112]),
  transforms.ToTensor(),
  transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
  ])

data_transforms_rand3 = transforms.Compose([
  transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
  transforms.Resize([48, 48]),  
  # transforms.CenterCrop(112*0.8),
  # transforms.Resize([112, 112]),
  transforms.ToTensor(),
  transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
  ])


# data_transforms_rand_old = transforms.Compose([
#   transforms.RandomRotation(degrees=15),
#   transforms.RandomAutocontrast(),
#   transforms.RandomAdjustSharpness(4),
#   transforms.Resize([112, 112]),
#   transforms.GaussianBlur(kernel_size=(7, 7), sigma=(2, 4)),
#   # transforms.CenterCrop(112*0.8),
#   # transforms.Resize([112, 112]),
#   transforms.ToTensor(),
#   transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
#   ])

# CenterCrop(size) #Crops the given image at the center.transforms.CenterCrop((40, 40)),
# ColorJitter([brightness, contrast, …]) #Randomly change the brightness, contrast, saturation and hue of an image.
# Grayscale([num_output_channels]) #Convert image to grayscale.


# RandomRotation 
#transform = transforms.RandomRotation(180) #-degree, +degree 사이만큼 회전
#ramdomaffine(회전+기울이기 옵션)
#transform = transforms.RandomAffine(180, shear=20)

# RandomApply(확률에 따라 여러개가 적용될수도 )
# transform = transforms.RandomApply([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.RandomRotation(180),
# ], p=0.9)
# RandomChoice (하나만 랜덤 선택)
# transform = transforms.RandomChoice([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.RandomRotation(180),
# ])

# RandomOrder #주어진 transform들을 모두 적용하는데, 그 순서를 랜덤하게 설정한다.
#내부적으로 random.shuffle을 사용하고 있다.
# transform = transforms.RandomOrder([
#     transforms.RandomHorizontalFlip(),
#     transforms.RandomVerticalFlip(),
#     transforms.RandomRotation(180),
#     transforms.RandomCrop((60, 60)),
# ])

# RandomGrayscale #ransform = transforms.RandomGrayscale(p=0.75)
# RandomAdjustSharpness0은 blurred, 1은 원본, 2부터는 선명한 정도를 2의 배수만큼 적용하는 것 같다.
# RandomAutocontrast
# RandomEqualize
#transform = transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET) 자동으로 augmentation

# adjust_brightness(img, brightness_factor)
# adjust_contrast(img, contrast_factor)
# adjust_gamma(img, gamma[, gain])
# adjust_sharpness(img, sharpness_factor)


In [ ]:
print(os.getcwd())
os.listdir()

In [ ]:
# Defining hyperparameters

BATCH_SIZE = 256
learning_rate = 0.001
EPOCHS = 15
numClasses = 43

In [ ]:
# Define path of training data

train_data_path = "./Train"
train_data_all = torchvision.datasets.ImageFolder(root = train_data_path, transform = data_transforms)
print(f"Number of training samples = {len(train_data_all)}")

In [ ]:
import sympy

mul_num = 3
lst_aug = [train_data_all]
for i in range(mul_num) :
  set_seed(i)
  rand_prinm = sympy.randprime(0, 10000)
  print(str(i) + ' : ' + str(rand_prinm))#, end = ', ')
  set_seed(rand_prinm)
  if i ==1 :
      train_data_aug = torchvision.datasets.ImageFolder(root = train_data_path, transform = data_transforms_rand2)    
  else:
      train_data_aug = torchvision.datasets.ImageFolder(root = train_data_path, transform = data_transforms_rand)
  lst_aug.append(train_data_aug)
set_seed()

In [ ]:
train_data_aug = torch.utils.data.ConcatDataset(lst_aug)

In [ ]:
# Divide data into training and validation (0.8 and 0.2)
ratio = 0.8
n_train_examples = int(len(train_data_aug) * ratio)
n_val_examples = len(train_data_aug) - n_train_examples

train_data, val_data = data.random_split(train_data_aug, [n_train_examples, n_val_examples])

print(f"Number of training samples = {len(train_data)}")
print(f"Number of validation samples = {len(val_data)}")

In [ ]:
## 너무 오래걸림 ㅠ
# # Plot histogram for training and validation data


# train_hist = [0]*numClasses
# for i, value in enumerate(train_data):
#     data, label = value
#     train_hist[int(label)] += 1
    
# val_hist = [0]*numClasses
# for i, value in enumerate(val_data):
#     data, label = value
#     train_hist[int(label)] += 1

# plt.bar(range(numClasses), train_hist, label="train")
# # plt.bar(range(numClasses), val_hist, label="val")
# legend = plt.legend(loc='upper right', shadow=True)
# plt.title("Distribution Plot")
# plt.xlabel("Class ID")
# plt.ylabel("# of examples")

# plt.savefig("train_val_split.png", bbox_inches = 'tight', pad_inches=0.5)

In [ ]:

# Create data loader for training and validation

# DataLoader(
#     train_dataset,
#     batch_size=batch_size,
#     num_workers=num_workers,
#     worker_init_fn=seed_worker
# )
train_aug_loader = DataLoader(train_data_aug, shuffle=True, batch_size = BATCH_SIZE)
train_loader = DataLoader(train_data, shuffle=True, batch_size = BATCH_SIZE)
val_loader = DataLoader(val_data, shuffle=True, batch_size = BATCH_SIZE)

# print(train_data_all[0][0].shape)
# print(train_data_all[0][1])
# print(len(train_data_all.samples))
# print(len(train_data_all.targets))
# print(len(train_data_all.classes))

In [ ]:
# Test_Label
test_ans = pd.read_csv('Test_Label.csv')

# Create Folder
os.mkdir('./Test_ans/')
for i in range(43) : 
  path = './Test_ans/'+str(i)
  if os.path.isdir(path) : 
    # os.rmdir(path)
    print('existing' + path)
  else : 
    os.mkdir(path)

# file path move for answer data validation

for f in  test_ans.iterrows():
  filename = f[1]['Filename']
  src = './Test/'
  dir = './Test_ans/'+str(f[1]['ClassId']) + '/'
  shutil.copy(src + filename, dir + filename)

In [ ]:
  # Define path of answer data
answer_data_path = "./Test_ans"
answer_data = torchvision.datasets.ImageFolder(root = answer_data_path, transform = data_transforms)
print(f"Number of answer samples = {len(answer_data)}")
answer_loader = DataLoader(answer_data, shuffle=False, batch_size = BATCH_SIZE)
# answer_loader = DataLoader(answer_data, shuffle=False, batch_size = BATCH_SIZE)
# print(answer_data.classes)
# print(answer_data.class_to_idx)

In [ ]:
# Plot histogram for ans data

ans_hist = [0]*numClasses
for i, value in enumerate(answer_data):
    data, label = value
    ans_hist[label] += 1

plt.bar(range(numClasses), ans_hist, label="answer")
legend = plt.legend(loc='upper right', shadow=True)
plt.title("Distribution Plot")
plt.xlabel("Class ID")
plt.ylabel("# of examples")

plt.savefig("answer.png", bbox_inches = 'tight', pad_inches=0.5)

In [ ]:
train_img = []  # imagefolder dataset 메모리에 올리기 위해 extract list
train_lab = []  
for (data, label) in train_data_aug :
  # print(label, data.shape)
  # print(data.numpy().shape)
  train_img.append(data.numpy())
  train_lab.append(label)
  # break
train_img = np.array(train_img)
train_lab = np.array(train_lab)
print(train_img.shape)
print(train_lab.shape)
train_img.reshape(len(train_lab), -1).shape

In [ ]:
from imblearn.over_sampling import SMOTE, ADASYN

smote = SMOTE(random_state = 0, k_neighbors = 5)
image_smote, label_smote = smote.fit_resample(train_img.reshape(len(train_lab), -1), train_lab)
s = train_img.shape
image_smote = image_smote.reshape(-1, s[1], s[2], s[3])
print(image_smote.shape)
print(label_smote.shape)

In [ ]:
# from torch.utils.data import TensorDataset, DataLoader

image_tensor = torch.tensor(image_smote, dtype=torch.float32)#.to("cuda:0")
label_tensor = torch.tensor(label_smote, dtype=torch.int64)#.to("cuda:0")
# from torch.utils.data import TensorDataset, DataLoader
train_ds = TensorDataset(image_tensor, label_tensor)
print(type(train_ds))
train_ds_loader = DataLoader(train_ds, shuffle=True, batch_size = BATCH_SIZE)

In [ ]:
# Plot histogram for training and validation data

train_hist = [0]*numClasses
for i, value in enumerate(train_ds):
    data, label = value
    # print(i, int(label), data.shape)
    # break
    train_hist[int(label)] += 1

plt.bar(range(numClasses), train_hist, label="train")
legend = plt.legend(loc='upper right', shadow=True)
plt.title("Distribution Plot")
plt.xlabel("Class ID")
plt.ylabel("# of examples")

plt.savefig("train_smote_split.png", bbox_inches = 'tight', pad_inches=0.5)

In [ ]:
# for i, value in enumerate(train_ds):
#     data, label = value
#     print(label, data.shape)
#     break

# MicronNet

In [ ]:
nclasses = 43 # GTSRB as 43 classes

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 1, kernel_size=1)
        self.conv2 = nn.Conv2d(1, 29, kernel_size=5)
        self.maxpool2 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv3 = nn.Conv2d(29, 59, kernel_size=3)
        self.maxpool3 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv4 = nn.Conv2d(59, 74, kernel_size=3)
        self.maxpool4 = nn.MaxPool2d(3, stride=2 , ceil_mode=True)
        self.conv2_drop = nn.Dropout2d()
        self.conv3_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1184, 300)
        self.fc2 = nn.Linear(300, nclasses)
        self.conv0_bn = nn.BatchNorm2d(3)
        self.conv1_bn = nn.BatchNorm2d(1)
        self.conv2_bn = nn.BatchNorm2d(29)
        self.conv3_bn = nn.BatchNorm2d(59)
        self.conv4_bn = nn.BatchNorm2d(74)
        self.dense1_bn = nn.BatchNorm1d(300)
    def forward(self, x):
        x =  F.relu(self.conv1_bn(self.conv1(self.conv0_bn(x))))
        x = F.relu(self.conv2_bn(self.conv2(x)))
        x = F.relu(self.conv3_bn(self.conv3( self.maxpool2(x))))
        x = F.relu(self.conv4_bn(self.conv4( self.maxpool3(x))))
        x = self.maxpool4(x)        
        x = x.view(-1, 1184)
        x = F.relu(self.fc1(x))
        x = self.dense1_bn(x)
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        x = F.dropout(x, training=self.training)
        return F.log_softmax(x, -1)

In [ ]:
epochs = 100
# epochs = 10000
batch_size = 50
seed = 1
log_interval=180
data = "data"
torch.manual_seed(1)
lr =0.007
momentum = 0.9
# step = 1000
step = 50
decay =  0.9996
l2_norm = 0.00001
cuda = True
resume = False
# These may change as described in paper

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = Net()
if  cuda: 
    model.to(device)

if resume :
    state_dict = torch.load("model_28.pth")
    model.load_state_dict(state_dict) 

In [ ]:
# https://stackoverflow.com/questions/65318064/can-i-trainoptimize-on-f1-score-loss-with-pytorch
# https://stackoverflow.com/questions/53354176/how-to-use-f-score-as-error-function-to-train-neural-networks
# loss <-- F-Measure as the Error Function to Train Neural Networks 
#

def validation():
    model.eval()
    validation_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in answer_loader:
            # data, target = Variable(data), Variable(target)
            data = data.to(device)
            target =target.to(device)
            output = model(data)
            validation_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        validation_loss /= len(val_loader.dataset)
        print(f"\nValidation set: Average loss: {format(validation_loss, '.4f')}, Accuracy: {correct}/{len(answer_loader.dataset)} ({format(100. * correct / len(answer_loader.dataset), '.0f')}%)\n")
    return validation_loss


def train(epoch , train_loader):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target).cuda()
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print(f"Train Epoch: {epoch}\t[{batch_idx * len(data)}/{len(train_loader.dataset)}\t({format(100. * batch_idx / len(train_loader),'.0f')}%)]\tLoss: {format(loss.item(),'.6f')}")


In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=lr , momentum=momentum, weight_decay=l2_norm, nesterov=True)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay)
# train_ds_loader train_aug_loader
temp = 10
epochs = 20
for epoch in range(1, epochs):
    train(epoch, train_aug_loader)
    val = validation()
    if epoch % step == 0 :
        scheduler.step()
    if val < temp : 
        temp = val
        model_file = 'model_' + str(epoch) + '.pth'
        torch.save(model.state_dict(), model_file)
        print('\nSaved model to ' + model_file + '. You can run `python evaluate.py ' + model_file + '` to generate the Kaggle formatted csv file')

In [ ]:
# Print confusion matrix

def plot_confusion_matrix(labels, pred_labels, classes):
    
    fig = plt.figure(figsize = (20, 20));
    ax = fig.add_subplot(1, 1, 1);
    cm = confusion_matrix(labels, pred_labels);
    cm = ConfusionMatrixDisplay(cm, display_labels = classes);
    cm.plot(values_format = 'd', cmap = 'Blues', ax = ax)
    plt.xticks(rotation = 20)
    
labels_arr = range(0, nclasses)
# plot_confusion_matrix(labels_list, y_pred_list, labels_arr)

In [ ]:
# Load the saved model

model = Net()
model.load_state_dict(torch.load(model_file)) # 'model_35.pth'
model = model.cuda()

# model = Net()
# model.load_state_dict(torch.load('model_35.pth')) # 'model_35.pth'
# model = model.cuda()

In [ ]:
test_answer = pd.read_csv('Test_Label.csv')
test_answer["pred_class"] = -1

num = range(nclasses)
labels = []
for i in num:
    labels.append(str(i))
labels = sorted(labels)
for i in num:
    labels[i] = int(labels[i])


test_transforms = transforms.Compose([
    transforms.Resize([48, 48]),
    transforms.ToTensor(),
    transforms.Normalize([meanR, meanG, meanB], [stdR, stdG, stdB])
])


# Perform classification

y_pred_list = []
corr_classified = 0
numExamples = len(test_answer)

##
labels_list = []
wronng_list = []


with torch.no_grad():
    model.eval()
    for f_name in test_answer["Filename"]:
        image = Image.open(f"Test//{f_name}")

        image = test_transforms(image)
        image = torch.unsqueeze(image, 0)
        # image = Variable(image)
        image = image.to(device)

        y_test_pred = model(image)
        y_pred_tags = y_test_pred.data.max(1, keepdim=True)[1]
        y_pred_tags = y_pred_tags.cpu().numpy()
        # print(y_pred_tags)

        y_pred = y_pred_tags[0][0]
        y_pred = labels[y_pred]
        
        y_pred_list.append(y_pred)

        # test_answer.loc[test_answer[test_answer["Filename"] == f_name].index, "pred_class"] = y_pred
        if test_answer[test_answer["Filename"] == f_name]["ClassId"].values[0] == y_pred:
            corr_classified += 1
        else : 
            wronng_list.append([f_name,test_answer[test_answer["Filename"] == f_name]["ClassId"].values[0], y_pred])
        # i += 1

df = pd.DataFrame(wronng_list) ## 데이터프래임 생성
df.to_csv('wronng_list.csv',index=False) ## 구분자를 탭으로 하여 저장. 인덱스칼럼은 저장 안함.

print("Number of correctly classified images = %d" % corr_classified)
print("Number of incorrectly classified images = %d" % (numExamples - corr_classified))
print("Final accuracy = %f" % (corr_classified / numExamples))

# Print classification report
print(classification_report(test_answer.ClassId, y_pred_list, digits=5))

plot_confusion_matrix(test_answer.ClassId, y_pred_list, labels_arr)

# AlexnetTS

In [ ]:
# # Function to count the number of parameters in the model

# def count_parameters(model):
#     return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:

# import torch.nn as nn

# class AlexnetTS(nn.Module):
#     def __init__(self, output_dim):
#         super().__init__()
        
#         self.features = nn.Sequential(
#             nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, stride=2, padding=1),
#             nn.MaxPool2d(kernel_size=2),
#             nn.ReLU(inplace=True),
            
#             nn.Conv2d(in_channels=64, out_channels=192, kernel_size=3, padding=1),
#             nn.MaxPool2d(kernel_size=2),
#             nn.ReLU(inplace=True),
            
#             nn.Conv2d(in_channels=192, out_channels=384, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
            
#             nn.Conv2d(in_channels=384, out_channels=256, kernel_size=3, padding=1),
#             nn.ReLU(inplace=True),
            
#             nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1),
#             nn.MaxPool2d(kernel_size=2),
#             nn.ReLU(inplace=True),
#             )
        
#         self.classifier = nn.Sequential(
#             nn.Dropout(0.5),
#             nn.Linear(256*7*7, 1000),
#             nn.ReLU(inplace=True),
            
#             nn.Dropout(0.5),
#             nn.Linear(in_features=1000, out_features=256),
#             nn.ReLU(inplace=True),
            
#             nn.Linear(256, output_dim)
#             )
        
#     def forward(self, x):
#         x = self.features(x)
#         h = x.view(x.shape[0], -1)
#         x = self.classifier(h)
#         return x, h

In [ ]:
# # Initialize the model
# # The model is defined in the class AlexnetTS in the file class_alexnetTS.py

# model = AlexnetTS(numClasses)
# print(f'The model has {count_parameters(model):,} trainable parameters')

# # Define optimizer and criterion functions

# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# criterion = nn.CrossEntropyLoss()

# # If CUDA is available, convert model and loss to cuda variables

# if torch.cuda.is_available():
#     model = model.cuda()
#     criterion = criterion.cuda()

# # Print model
# print(model)

# # Print summary of the model for the given dimension of the image
# print(summary(model, (3, 48, 48))) 

# # Print model's state dict
# print("Model's state dict:")
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())
# print("")

# # Print optimizer details

# print("Optimizer details:")
# print(optimizer)
# print("")

In [ ]:
# # Function to calculate accuracy

# def calculate_accuracy(y_pred, y):
#     top_pred = y_pred.argmax(1, keepdim = True)
#     correct = top_pred.eq(y.view_as(top_pred)).sum()
#     acc = correct.float() / y.shape[0]
#     return acc

In [ ]:
# # Function to perform training of the model

# def train(model, loader, opt, criterion):
#     epoch_loss = 0
#     epoch_acc = 0
    
#     # Train the model
#     model.train()
    
#     for (images, labels) in loader:
#         images = images.cuda()
#         labels = labels.cuda()
#         # labels = labels.cuda().type(torch.LongTensor)
        
#         # Training pass
#         opt.zero_grad()
        
#         output, _ = model(images)
#         loss = criterion(output, labels)
        
#         # Backpropagation
#         loss.backward()
        
#         # Calculate accuracy
#         acc = calculate_accuracy(output, labels)
        
#         # Optimizing weights
#         opt.step()
        
#         epoch_loss += loss.item()
#         epoch_acc += acc.item()
        
#     return epoch_loss / len(loader), epoch_acc / len(loader)

In [ ]:

# # Function to perform evaluation on the trained model

# def evaluate(model, loader, opt, criterion):
#     epoch_loss = 0
#     epoch_acc = 0
    
#     # Evaluate the model
#     model.eval()
    
#     with torch.no_grad():
#         for (images, labels) in loader:
#             images = images.cuda()
#             labels = labels.cuda()
            
#             # Run predictions
#             output, _ = model(images)
#             loss = criterion(output, labels)
            
#             # Calculate accuracy
#             acc = calculate_accuracy(output, labels)
            
#             epoch_loss += loss.item()
#             epoch_acc += acc.item()
    
#     return epoch_loss / len(loader), epoch_acc / len(loader)

In [ ]:
# # Perform training
# local_epoch = 50
# # List to save training and val loss and accuracies
# # train_loss_list = [0]*EPOCHS
# # train_acc_list = [0]*EPOCHS
# # val_loss_list = [0]*EPOCHS
# # val_acc_list = [0]*EPOCHS
# train_loss_list = [0]*local_epoch
# train_acc_list = [0]*local_epoch
# val_loss_list = [0]*local_epoch
# val_acc_list = [0]*local_epoch

# # for epoch in range(EPOCHS):
# for epoch in range(local_epoch):
#     print("Epoch-%d: " % (epoch))

#     train_start_time = time.monotonic()
#     # train_loss, train_acc = train(model, answer_loader, optimizer, criterion)
#     # train_loss, train_acc = train(model, train_all_loader, optimizer, criterion)
#     train_loss, train_acc = train(model, train_aug_loader, optimizer, criterion)
#     train_end_time = time.monotonic()

#     val_start_time = time.monotonic()
#     # val_loss, val_acc = evaluate(model, val_loader, optimizer, criterion) # val data
#     val_loss, val_acc = evaluate(model, answer_loader, optimizer, criterion) # answer data
#     val_end_time = time.monotonic()
    
#     train_loss_list[epoch] = train_loss
#     train_acc_list[epoch] = train_acc
#     val_loss_list[epoch] = val_loss
#     val_acc_list[epoch] = val_acc
    
#     print("Training: Loss = %.4f, Accuracy = %.4f, Time = %.2f seconds" % (train_loss, train_acc, train_end_time - train_start_time))
#     print("Validation: Loss = %.4f, Accuracy = %.4f, Time = %.2f seconds" % (val_loss, val_acc, val_end_time - val_start_time))
#     print("")

In [ ]:
# # Saving the model

# # Create folder to save model
# MODEL_FOLDER = "./Model"
# if not os.path.isdir(MODEL_FOLDER):
#     os.mkdir(MODEL_FOLDER)
    
# PATH_TO_MODEL = MODEL_FOLDER + "/pytorch_classification_alexnetTS.pth"
# if os.path.exists(PATH_TO_MODEL):
#     os.remove(PATH_TO_MODEL)
# torch.save(model.state_dict(), PATH_TO_MODEL)

# print("Model saved at %s" %(PATH_TO_MODEL))

In [ ]:
# # Plot loss and accuracies for training and validation data

# _, axs = plt.subplots(1, 2, figsize=(15, 5))

# # Loss plot
# axs[0].plot(train_loss_list, label="train")
# axs[0].plot(val_loss_list, label="val")
# axs[0].set_title("Plot - Loss")
# axs[0].set_xlabel("Epochs")
# axs[0].set_ylabel("Loss")
# legend = axs[0].legend(loc='upper right', shadow=False)

# # Accuracy plot
# axs[1].plot(train_acc_list, label="train")
# axs[1].plot(val_acc_list, label="val")
# axs[1].set_title("Plot - Accuracy")
# axs[1].set_xlabel("Epochs")
# axs[1].set_ylabel("Accuracy")
# legend = axs[1].legend(loc='center right', shadow=True)

In [ ]:
# test_answer = pd.read_csv('Test_Label.csv')
# test_answer["pred_class"] = -1

# num = range(numClasses)
# labels = []
# for i in num:
#     labels.append(str(i))
# labels = sorted(labels)
# for i in num:
#     labels[i] = int(labels[i])

# # Perform classification
# # test_transforms = transforms.Compose([
# #     transforms.Resize([112, 112]),
# #     transforms.ToTensor()
# #     ])
# test_transforms = transforms.Compose([
#     transforms.Resize([48, 48]),
#     transforms.ToTensor(),
#     # transforms.Normalize(mean=1, std=0.5)
#     ])
        
# y_pred_list = []
# corr_classified = 0
# numExamples = len(test_answer)

# ##
# labels_list = []

# with torch.no_grad():
#     model.eval()
#     for f_name in test_answer["Filename"]:
#         image = Image.open(f"Test//{f_name}")

#         image = test_transforms(image)
#         image = torch.unsqueeze(image, 0)
#         image = image.cuda()

#         y_test_pred = model(image)

#         y_pred_softmax = torch.log_softmax(y_test_pred[0], dim=1)
#         _, y_pred_tags = torch.max(y_pred_softmax, dim=1)
#         y_pred_tags = y_pred_tags.cpu().numpy()

#         y_pred = y_pred_tags[0]
#         y_pred = labels[y_pred]
        
#         y_pred_list.append(y_pred)

#         test_answer.loc[test_answer[test_answer["Filename"] == f_name].index, "pred_class"] = y_pred
#         if test_answer[test_answer["Filename"] == f_name]["ClassId"].values[0] == y_pred:
#             corr_classified += 1

#         # i += 1

# print("Number of correctly classified images = %d" % corr_classified)
# print("Number of incorrectly classified images = %d" % (numExamples - corr_classified))
# print("Final accuracy = %f" % (corr_classified / numExamples))

# # Print classification report
# print(classification_report(test_answer.ClassId, test_answer.pred_class))

# # plot_confusion_matrix(test_answer.ClassId, test_answer.pred_class, labels_arr)

In [ ]:
# Number of correctly classified images = 8452
# Number of incorrectly classified images = 218
# Final accuracy = 0.974856
#               precision    recall  f1-score   support

#            0       1.00      1.00      1.00        60
#            1       0.99      0.99      0.99       420
#            2       0.97      0.97      0.97       420
#            3       0.96      0.99      0.97       420
#            4       1.00      0.98      0.99       420
#            5       0.98      0.97      0.97       420
#            6       1.00      0.87      0.93        60
#            7       0.99      1.00      1.00       420
#            8       1.00      0.99      0.99       420
#            9       1.00      1.00      1.00       420
#           10       1.00      0.99      1.00       420
#           11       0.94      0.99      0.97       420
#           12       0.99      0.98      0.99       420
#           13       0.99      0.99      0.99       420
#           14       1.00      1.00      1.00       210
#           15       0.99      0.99      0.99       210
#           16       1.00      1.00      1.00        60
#           17       1.00      1.00      1.00       210
#           18       0.98      0.87      0.92       210
#           19       0.98      1.00      0.99        60
#           20       0.92      0.98      0.95        60
#           21       0.98      0.75      0.85        60
#           22       1.00      0.98      0.99        60
#           23       1.00      0.80      0.89        60
#           24       0.97      0.95      0.96        60
#           25       0.99      0.98      0.98       420
#           26       1.00      0.92      0.96        60
#           27       0.79      0.55      0.65        60
#           28       0.78      0.98      0.87        60
#           29       0.88      1.00      0.94        60
#           30       0.61      0.58      0.60        60
#           31       0.90      0.99      0.94       210
#           32       0.83      1.00      0.91        60
#           33       0.99      0.99      0.99       210
#           34       1.00      1.00      1.00        60
#           35       1.00      1.00      1.00       210
#           36       0.98      1.00      0.99        60
#           37       1.00      1.00      1.00        60
#           38       1.00      0.99      0.99       420
#           39       0.94      0.98      0.96        60
#           40       1.00      0.97      0.98        60
#           41       0.79      0.93      0.85        60
#           42       1.00      1.00      1.00        60

#     accuracy                           0.97      8670
#    macro avg       0.96      0.95      0.95      8670
# weighted avg       0.98      0.97      0.97      8670

In [ ]:
# test_answer = pd.read_csv('Test_Label.csv')
# test_answer["pred_class"] = -1

# num = range(numClasses)
# labels = []
# for i in num:
#     labels.append(str(i))
# labels = sorted(labels)
# for i in num:
#     labels[i] = int(labels[i])

# # Perform classification
# test_transforms = transforms.Compose([
#     transforms.Resize([48, 48]),
#     transforms.ToTensor(),
#     # transforms.Normalize(mean=1, std=0.5)
#     ])
        
# y_pred_list = []
# corr_classified = 0
# numExamples = len(test_answer)

# ##
# labels_list = []
# wronng_list = []
# with torch.no_grad():
#     model.eval()
#     for f_name in test_answer["Filename"]:
#         image = Image.open(f"Test//{f_name}")

#         image = test_transforms(image)
#         image = torch.unsqueeze(image, 0)
#         image = image.cuda()

#         y_test_pred = model(image)

#         y_pred_softmax = torch.log_softmax(y_test_pred[0], dim=1)
#         _, y_pred_tags = torch.max(y_pred_softmax, dim=1)
#         y_pred_tags = y_pred_tags.cpu().numpy()

#         y_pred = y_pred_tags[0]
#         y_pred = labels[y_pred]
        
#         y_pred_list.append(y_pred)

#         test_answer.loc[test_answer[test_answer["Filename"] == f_name].index, "pred_class"] = y_pred
#         if test_answer[test_answer["Filename"] == f_name]["ClassId"].values[0] == y_pred:
#             corr_classified += 1
#         else : 
#           wronng_list.append([f_name,test_answer[test_answer["Filename"] == f_name]["ClassId"].values[0], y_pred])
#         # i += 1

# print("Number of correctly classified images = %d" % corr_classified)
# print("Number of incorrectly classified images = %d" % (numExamples - corr_classified))
# print("Final accuracy = %f" % (corr_classified / numExamples))

# # Print classification report
# print(classification_report(test_answer.ClassId, test_answer.pred_class))

# # plot_confusion_matrix(test_answer.ClassId, test_answer.pred_class, labels_arr)

In [ ]:
# df = pd.DataFrame(wronng_list) ## 데이터프래임 생성
# df.to_csv('wronng_list.csv',index=False) ## 구분자를 탭으로 하여 저장. 인덱스칼럼은 저장 안함.

In [ ]:
# # Print confusion matrix

# def plot_confusion_matrix(labels, pred_labels, classes):
    
#     fig = plt.figure(figsize = (20, 20));
#     ax = fig.add_subplot(1, 1, 1);
#     # print(labels)
#     # print(pred_labels)
#     cm = confusion_matrix(labels, pred_labels);
#     cm = ConfusionMatrixDisplay(cm, display_labels = classes);
#     cm.plot(values_format = 'd', cmap = 'Blues', ax = ax)
#     plt.xticks(rotation = 20)
    
# labels_arr = range(0, numClasses)
# # plot_confusion_matrix(test_answer.ClassId, test_answer.pred_class, labels_arr)
# plot_confusion_matrix(test_answer.ClassId, y_pred_list, labels_arr)


# test using val_data

In [ ]:
# val_data.dataset

In [ ]:
# # Load the saved model

# model = AlexnetTS(numClasses)
# model.load_state_dict(torch.load(PATH_TO_MODEL))
# model = model.cuda()

In [ ]:
# # Perform classification

# y_pred_list = []
# corr_classified = 0
# # numExamples = len(val_data)
# numExamples = len(val_data)

# ##
# labels_list = []

# with torch.no_grad():
#     model.eval()

#     # i = 0

#     for image, label_true in data.DataLoader(val_data, shuffle=True, batch_size = 1): # shuffle = False when test
#         labels_list.append(label_true.cpu().numpy()[0])
#         image = image.cuda()

#         y_test_pred = model(image)

#         y_pred_softmax = torch.log_softmax(y_test_pred[0], dim=1)
#         _, y_pred_tags = torch.max(y_pred_softmax, dim=1)
#         y_pred_tags = y_pred_tags.cpu().numpy()

#         y_pred_list.append(y_pred_tags[0])
#         if label_true == y_pred_tags[0]:
#             corr_classified += 1

#         # i += 1

# print("Number of correctly classified images = %d" % corr_classified)
# print("Number of incorrectly classified images = %d" % (numExamples - corr_classified))
# print("Final accuracy = %f" % (corr_classified / numExamples))

In [ ]:
# # Print classification report

# print(classification_report(labels_list, y_pred_list))

In [ ]:
# # Print confusion matrix

# def plot_confusion_matrix(labels, pred_labels, classes):
    
#     fig = plt.figure(figsize = (20, 20));
#     ax = fig.add_subplot(1, 1, 1);
#     cm = confusion_matrix(labels, pred_labels);
#     cm = ConfusionMatrixDisplay(cm, display_labels = classes);
#     cm.plot(values_format = 'd', cmap = 'Blues', ax = ax)
#     plt.xticks(rotation = 20)
    
# labels_arr = range(0, numClasses)
# plot_confusion_matrix(labels_list, y_pred_list, labels_arr)
